## Name : Darpan Gaur
## Roll no : CO21BTECH11004

# Worksheet1 — Allen–Cahn + Cahn–Hilliard Coupled System

**Instructions**: Use the provided simulation code to answer the following questions. Include code cells, plots, and explanations as needed.

### Question 1

1. Compare the total interfacial area (points with 0.1 < phi < 0.9) over time for two cases: 100 randomly placed particles vs. 20 large ones in a grid. What physical insights can you draw from the trends?

---

*Your response below:*  </br>
</br>
Timestamps is fixed to 10000, for both simulation. </br>
$\textbf{Case 1: 100 random particles}$
- Radius : 5
- Area with phi > 0.1 :  12535
- Area with phi > 0.9 :  7218
- Desired Area phi > 0.1 and phi < 0.9 :  5317


$\textbf{Case 1: 20 random particles large particles}$
- Radius : 11
- Area with phi > 0.1 :  13124
- Area with phi > 0.9 :  8696
- Desired Area phi > 0.1 and phi < 0.9 :  4428

$\textbf{Observation and thoughts}$

- Interfacial area is greater for 100 particles as compared to 20 large particles.
- Total initial area $100 * \pi * r^2$, for 100 particles. 
- Total inital area $20 * \pi * (2r)^2$ for 20 particles. So kind of similar inital areas with radius variations for both cases.
- Intial area where phi>0.1 : 7404 for 100 particles, while inital area where phi>0.1 : 7540 for 20 large particles, as radius is taken twice of initial.
- The final interfacial area for 100 particles is large as compared to 20 particles.
- This is due as small particles shinking, and adding to large paricles, it takes less time as they are more and near, as compared to large particles.
- Number of particles decreased rapidly in case 1, as compared to case 2.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Input parameters

Nx = 256
Ny = 256
dx = 1.0
dy = 1.0
dt = 0.1

## Model parameters 

omega = 1.0
kappa_c = 1.0 #Cahn-Hilliard
kappa_phi = 1.0 #Allen-Cahn 
c0 = 0.1
cAlpha_eq = 0.0
cBeta_eq = 1.0
A = 1.0
B = 1.0
M = 1.0
L = 1.0

# Initialize fields
phi = np.zeros((Nx, Ny))
comp = np.full((Nx, Ny), c0)

# # Particle radius and variation
# base_radius = 3.5
# radius_variation = 0.01
# safety_margin = 2.0  # Ensure no overlap within 2 times the radius

# # Seed the random number generator for reproducibility

# np.random.seed(11)
# particle_positions = []
# radii = []

# while len(particle_positions) < 100:
#     radius = base_radius * (1 + np.random.uniform(-radius_variation, radius_variation))
#     x, y = np.random.randint(0, Nx), np.random.randint(0, Ny)

#     # Check for overlap with twice the radius
#     if all(np.sqrt(((x - px) % Nx)**2 + ((y - py) % Ny)**2) >= (r + radius) * safety_margin
#            for px, py, r in particle_positions):
#         particle_positions.append((x, y, radius))
#         radii.append(radius)

# # Place particles on the grid
# for (x, y, radius) in particle_positions:
#     for i in range(-int(radius), int(radius) + 1):
#         for j in range(-int(radius), int(radius) + 1):
#             if i**2 + j**2 <= radius**2:
#                 xi = (x + i) % Nx  # Apply periodic boundary conditions
#                 yj = (y + j) % Ny
#                 phi[xi, yj] = 1.0
#                 comp[xi, yj] = cBeta_eq

# Particle radius and variation
base_radius = 5
radius_variation = 0.01

# Seed the random number generator for reproducibility
np.random.seed(42)
particle_positions = []
radii = []

# Function to calculate the correct distance considering periodic boundaries
def check_distance(x1, y1, r1, x2, y2, r2, Nx, Ny):
    dx = min(abs(x1 - x2), Nx - abs(x1 - x2))
    dy = min(abs(y1 - y2), Ny - abs(y1 - y2))
    distance = np.sqrt(dx**2 + dy**2)
    return distance >= (r1 + r2) * 2  # Safety margin of two times the radius

# Generate 100 unique particles ensuring no overlap within twice the radius
while len(particle_positions) < 100:
    radius = base_radius * (1 + np.random.uniform(-radius_variation, radius_variation))
    x, y = np.random.randint(0, Nx), np.random.randint(0, Ny)

    if all(check_distance(x, y, radius, px, py, r, Nx, Ny) for px, py, r in particle_positions):
        particle_positions.append((x, y, radius))
        radii.append(radius)

# Place particles on the grid
for (x, y, radius) in particle_positions:
    for i in range(-int(radius), int(radius) + 1):
        for j in range(-int(radius), int(radius) + 1):
            if i**2 + j**2 <= radius**2:
                xi = (x + i) % Nx  # Apply periodic boundary conditions
                yj = (y + j) % Ny
                phi[xi, yj] = 1.0
                comp[xi, yj] = cBeta_eq

X,Y = np.meshgrid(range(Nx),range(Ny)) 
fig = plt.figure()
ax = fig.add_subplot()
plt.contourf(X,Y,phi,cmap = 'jet')
ax.set_aspect('equal',adjustable='box')
plt.colorbar()
plt.show()

phi_new = phi
comp_new = comp
#delkx is grid spacing along kx in Fourier space
#delky is grid spacing along ky in Fourier space
delkx = 2*np.pi/(Nx*dx)
delky = 2*np.pi/(Ny*dy)



#Periodic boundar condition in fourier space
kx = np.zeros(Nx)
for i in range(Nx):
    if i < Nx/2:
        kx[i] = i * delkx
    else:
        kx[i] = (i-Nx)*delkx
        
ky = np.zeros(Ny)
for i in range(Ny):
    if i < Ny/2:
        ky[i] = i * delky
    else:
        ky[i] = (i-Ny)*delky
        
kpow2 = np.zeros([Nx,Ny])
kpow4 = np.zeros([Nx,Ny])
for i in range(Nx):
    for j in range(Ny):
        kpow2[i,j] = kx[i]**2 + ky[j]**2
        kpow4[i,j] = kpow2[i,j] * kpow2[i,j]

for n in range(10000):
    hphi = np.multiply(phi**3,10. - 15. * phi + 6. * phi**2)
    gphi = omega * np.multiply(phi**2,(1-phi)**2)
    hprime = 30.0 * np.multiply(phi**2,(1-phi)**2)
    gprime = 2.0*omega * np.multiply(phi - phi**2,1.0-2.0*phi)
    fAlpha = A * (comp - cAlpha_eq)**2
    fBeta  = B * (cBeta_eq - comp)**2  
    
    dfdc  = 2.0 * A * np.multiply(1.0-hphi,comp-cAlpha_eq)  - 2.0 * B * np.multiply(hphi,cBeta_eq-comp)
    dfdphi = np.multiply(hprime,fBeta-fAlpha) + omega * gprime
    
    
    dfdchat = np.fft.fft2(dfdc)
    dfdphihat = np.fft.fft2(dfdphi)
    
    phi_hat = np.fft.fft2(phi_new)
    phi_hat = (phi_hat - L*dt*dfdphihat)/(1+2*L*kappa_phi*kpow2*dt)
    phi_new = np.fft.ifft2(phi_hat).real
    phi = np.copy(phi_new)
    
    comp_hat = np.fft.fft2(comp_new)
    comp_hat = (comp_hat - M*dt*kpow2*dfdchat)/(1+2.0*M*kappa_c*kpow4*dt)
    comp_new = np.fft.ifft2(comp_hat).real
    comp = np.copy(comp_new)

    # if n%200==0:
    #     print(comp_new.max())
    #     fnam = str(n).zfill(5)
    #     X,Y = np.meshgrid(range(Nx),range(Ny))        
        
    #     fig = plt.figure()
    #     ax = fig.add_subplot()
    #     plt.contourf(X,Y,comp_new,cmap = 'jet')
    #     plt.colorbar()
    #     ax.set_aspect('equal',adjustable='box')
    #     #plt.savefig('fig' + fnam + '.jpg')
    #     plt.show()
X, Y = np.meshgrid(range(Nx), range(Ny))
fig = fig.add_subplot()
plt.contourf(X,Y,comp_new,cmap = 'jet')
plt.colorbar()
ax.set_aspect('equal',adjustable='box')
plt.show()

In [ ]:
gr1 = np.sum(phi>0.1)
gr2 = np.sum(phi>0.9)
desired = gr1-gr2
print(f"Area with phi > 0.1 : ", gr1)
print(f"Area with phi > 0.9 : ", gr2)
print(f"Desired Area phi > 0.1 and phi < 0.9 : ", gr1-gr2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Input parameters

Nx = 256
Ny = 256
dx = 1.0
dy = 1.0
dt = 0.1

## Model parameters 

omega = 1.0
kappa_c = 1.0 #Cahn-Hilliard
kappa_phi = 1.0 #Allen-Cahn 
c0 = 0.1
cAlpha_eq = 0.0
cBeta_eq = 1.0
A = 1.0
B = 1.0
M = 1.0
L = 1.0

# Initialize fields
phi = np.zeros((Nx, Ny))
comp = np.full((Nx, Ny), c0)

# # Particle radius and variation
# base_radius = 3.5
# radius_variation = 0.01
# safety_margin = 2.0  # Ensure no overlap within 2 times the radius

# # Seed the random number generator for reproducibility

# np.random.seed(11)
# particle_positions = []
# radii = []

# while len(particle_positions) < 100:
#     radius = base_radius * (1 + np.random.uniform(-radius_variation, radius_variation))
#     x, y = np.random.randint(0, Nx), np.random.randint(0, Ny)

#     # Check for overlap with twice the radius
#     if all(np.sqrt(((x - px) % Nx)**2 + ((y - py) % Ny)**2) >= (r + radius) * safety_margin
#            for px, py, r in particle_positions):
#         particle_positions.append((x, y, radius))
#         radii.append(radius)

# # Place particles on the grid
# for (x, y, radius) in particle_positions:
#     for i in range(-int(radius), int(radius) + 1):
#         for j in range(-int(radius), int(radius) + 1):
#             if i**2 + j**2 <= radius**2:
#                 xi = (x + i) % Nx  # Apply periodic boundary conditions
#                 yj = (y + j) % Ny
#                 phi[xi, yj] = 1.0
#                 comp[xi, yj] = cBeta_eq

# Particle radius and variation
base_radius = 11
radius_variation = 0.01

# Seed the random number generator for reproducibility
np.random.seed(42)
particle_positions = []
radii = []

# Function to calculate the correct distance considering periodic boundaries
def check_distance(x1, y1, r1, x2, y2, r2, Nx, Ny):
    dx = min(abs(x1 - x2), Nx - abs(x1 - x2))
    dy = min(abs(y1 - y2), Ny - abs(y1 - y2))
    distance = np.sqrt(dx**2 + dy**2)
    return distance >= (r1 + r2) * 2  # Safety margin of two times the radius

# Generate 100 unique particles ensuring no overlap within twice the radius
while len(particle_positions) < 20:
    radius = base_radius * (1 + np.random.uniform(-radius_variation, radius_variation))
    x, y = np.random.randint(0, Nx), np.random.randint(0, Ny)

    if all(check_distance(x, y, radius, px, py, r, Nx, Ny) for px, py, r in particle_positions):
        particle_positions.append((x, y, radius))
        radii.append(radius)

# Place particles on the grid
for (x, y, radius) in particle_positions:
    for i in range(-int(radius), int(radius) + 1):
        for j in range(-int(radius), int(radius) + 1):
            if i**2 + j**2 <= radius**2:
                xi = (x + i) % Nx  # Apply periodic boundary conditions
                yj = (y + j) % Ny
                phi[xi, yj] = 1.0
                comp[xi, yj] = cBeta_eq

X,Y = np.meshgrid(range(Nx),range(Ny)) 
fig = plt.figure()
ax = fig.add_subplot()
plt.contourf(X,Y,phi,cmap = 'jet')
ax.set_aspect('equal',adjustable='box')
plt.colorbar()
plt.show()

phi_new = phi
comp_new = comp
#delkx is grid spacing along kx in Fourier space
#delky is grid spacing along ky in Fourier space
delkx = 2*np.pi/(Nx*dx)
delky = 2*np.pi/(Ny*dy)



#Periodic boundar condition in fourier space
kx = np.zeros(Nx)
for i in range(Nx):
    if i < Nx/2:
        kx[i] = i * delkx
    else:
        kx[i] = (i-Nx)*delkx
        
ky = np.zeros(Ny)
for i in range(Ny):
    if i < Ny/2:
        ky[i] = i * delky
    else:
        ky[i] = (i-Ny)*delky
        
kpow2 = np.zeros([Nx,Ny])
kpow4 = np.zeros([Nx,Ny])
for i in range(Nx):
    for j in range(Ny):
        kpow2[i,j] = kx[i]**2 + ky[j]**2
        kpow4[i,j] = kpow2[i,j] * kpow2[i,j]

for n in range(10000):
    hphi = np.multiply(phi**3,10. - 15. * phi + 6. * phi**2)
    gphi = omega * np.multiply(phi**2,(1-phi)**2)
    hprime = 30.0 * np.multiply(phi**2,(1-phi)**2)
    gprime = 2.0*omega * np.multiply(phi - phi**2,1.0-2.0*phi)
    fAlpha = A * (comp - cAlpha_eq)**2
    fBeta  = B * (cBeta_eq - comp)**2  
    
    dfdc  = 2.0 * A * np.multiply(1.0-hphi,comp-cAlpha_eq)  - 2.0 * B * np.multiply(hphi,cBeta_eq-comp)
    dfdphi = np.multiply(hprime,fBeta-fAlpha) + omega * gprime
    
    
    dfdchat = np.fft.fft2(dfdc)
    dfdphihat = np.fft.fft2(dfdphi)
    
    phi_hat = np.fft.fft2(phi_new)
    phi_hat = (phi_hat - L*dt*dfdphihat)/(1+2*L*kappa_phi*kpow2*dt)
    phi_new = np.fft.ifft2(phi_hat).real
    phi = np.copy(phi_new)
    
    comp_hat = np.fft.fft2(comp_new)
    comp_hat = (comp_hat - M*dt*kpow2*dfdchat)/(1+2.0*M*kappa_c*kpow4*dt)
    comp_new = np.fft.ifft2(comp_hat).real
    comp = np.copy(comp_new)

    # if n%200==0:
    #     print(comp_new.max())
    #     fnam = str(n).zfill(5)
    #     X,Y = np.meshgrid(range(Nx),range(Ny))        
        
    #     fig = plt.figure()
    #     ax = fig.add_subplot()
    #     plt.contourf(X,Y,comp_new,cmap = 'jet')
    #     plt.colorbar()
    #     ax.set_aspect('equal',adjustable='box')
    #     #plt.savefig('fig' + fnam + '.jpg')
    #     plt.show()
X, Y = np.meshgrid(range(Nx), range(Ny))
fig = fig.add_subplot()
plt.contourf(X,Y,comp_new,cmap = 'jet')
plt.colorbar()
ax.set_aspect('equal',adjustable='box')
plt.show()

In [ ]:
gr1 = np.sum(phi>0.1)
gr2 = np.sum(phi>0.9)
desired = gr1-gr2
print(f"Area with phi > 0.1 : ", gr1)
print(f"Area with phi > 0.9 : ", gr2)
print(f"Desired Area phi > 0.1 and phi < 0.9 : ", gr1-gr2)

### Question 2

2. Change the interpolating function h(phi) to a linear form h(phi) = phi. How does this affect interface width, phase coupling, and overall stability? What would this imply for phase transitions in real materials?

---

*Your response below:* </br>
- Change h(phi) to linear h(phi) = phi.
- hprime = 1
- The size of particles increases, interface witdth decrease, as interface also have high contentration.
- As compared to previous h(phi) function there aer less particles vanishing in this case.
- It is still stable as solution is not exploding. Only thing : is it valid in real world. 

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Define input and model parameters
Nx = 256
Ny = 256
dx = 1.0
dy = 1.0
dt = 0.1

omega = 1.0
kappa_c = 1.0  # Cahn-Hilliard
kappa_phi = 1.0  # Allen-Cahn 
c0 = 0.1
cAlpha_eq = 0.0
cBeta_eq = 1.0
A = 1.0
B = 1.0
M = 1.0
L = 1.0

# Initialize fields
phi = np.zeros((Nx, Ny))
comp = np.full((Nx, Ny), c0)

# Particle radius and variation
base_radius = 5
radius_variation = 0.01

# Seed the random number generator for reproducibility
np.random.seed(42)
particle_positions = []
radii = []

# Function to calculate the correct distance considering periodic boundaries
def check_distance(x1, y1, r1, x2, y2, r2, Nx, Ny):
    dx = min(abs(x1 - x2), Nx - abs(x1 - x2))
    dy = min(abs(y1 - y2), Ny - abs(y1 - y2))
    distance = np.sqrt(dx**2 + dy**2)
    return distance >= (r1 + r2) * 2  # Safety margin of two times the radius

# Generate 100 unique particles ensuring no overlap within twice the radius
while len(particle_positions) < 100:
    radius = base_radius * (1 + np.random.uniform(-radius_variation, radius_variation))
    x, y = np.random.randint(0, Nx), np.random.randint(0, Ny)

    if all(check_distance(x, y, radius, px, py, r, Nx, Ny) for px, py, r in particle_positions):
        particle_positions.append((x, y, radius))
        radii.append(radius)

# Place particles on the grid
for (x, y, radius) in particle_positions:
    for i in range(-int(radius), int(radius) + 1):
        for j in range(-int(radius), int(radius) + 1):
            if i**2 + j**2 <= radius**2:
                xi = (x + i) % Nx  # Apply periodic boundary conditions
                yj = (y + j) % Ny
                phi[xi, yj] = 1.0
                comp[xi, yj] = cBeta_eq


X,Y = np.meshgrid(range(Nx),range(Ny)) 
fig = plt.figure()
ax = fig.add_subplot()
plt.contourf(X,Y,phi,cmap = 'jet')
ax.set_aspect('equal',adjustable='box')
plt.colorbar()
plt.show()

phi_new = phi
comp_new = comp
#delkx is grid spacing along kx in Fourier space
#delky is grid spacing along ky in Fourier space
delkx = 2*np.pi/(Nx*dx)
delky = 2*np.pi/(Ny*dy)



#Periodic boundar condition in fourier space
kx = np.zeros(Nx)
for i in range(Nx):
    if i < Nx/2:
        kx[i] = i * delkx
    else:
        kx[i] = (i-Nx)*delkx
        
ky = np.zeros(Ny)
for i in range(Ny):
    if i < Ny/2:
        ky[i] = i * delky
    else:
        ky[i] = (i-Ny)*delky
        
kpow2 = np.zeros([Nx,Ny])
kpow4 = np.zeros([Nx,Ny])
for i in range(Nx):
    for j in range(Ny):
        kpow2[i,j] = kx[i]**2 + ky[j]**2
        kpow4[i,j] = kpow2[i,j] * kpow2[i,j]


# Prepare figure for animation
fig, ax = plt.subplots()
c = ax.contourf(comp, levels=50, cmap='jet')
ax.set_aspect('equal')
colorbar = fig.colorbar(c, ax=ax)

# Animation update function
def update(frame):
    global phi, comp, phi_new, comp_new
    for _ in range(100):
        # hphi = np.multiply(phi**3,10. - 15. * phi + 6. * phi**2)
        hphi = phi
        gphi = omega * np.multiply(phi**2,(1-phi)**2)
        # hprime = 30.0 * np.multiply(phi**2,(1-phi)**2)
        hprime = 1
        gprime = 2.0*omega * np.multiply(phi - phi**2,1.0-2.0*phi)
        fAlpha = A * (comp - cAlpha_eq)**2
        fBeta  = B * (cBeta_eq - comp)**2  
    
        dfdc  = 2.0 * A * np.multiply(1.0-hphi,comp-cAlpha_eq)  - 2.0 * B * np.multiply(hphi,cBeta_eq-comp)
        dfdphi = np.multiply(hprime,fBeta-fAlpha) + omega * gprime
    
    
        dfdchat = np.fft.fft2(dfdc)
        dfdphihat = np.fft.fft2(dfdphi)
    
        phi_hat = np.fft.fft2(phi_new)
        phi_hat = (phi_hat - L*dt*dfdphihat)/(1+2*L*kappa_phi*kpow2*dt)
        phi_new = np.fft.ifft2(phi_hat).real
        phi = np.copy(phi_new)
    
        comp_hat = np.fft.fft2(comp_new)
        comp_hat = (comp_hat - M*dt*kpow2*dfdchat)/(1+2.0*M*kappa_c*kpow4*dt)
        comp_new = np.fft.ifft2(comp_hat).real
        comp = np.copy(comp_new)
    
    ax.clear()
    contour = ax.contourf(comp, 50, cmap='jet')
    ax.set_aspect('equal')
    return contour,

# from matplotlib.animation import PillowWriter

# Create and display the animation
ani = FuncAnimation(fig, update, frames=100, repeat=False)
# HTML(ani.to_html5_video())
HTML(ani.to_jshtml())

# Create and display the animation
# ani = FuncAnimation(fig, update, frames=100, repeat=False)
# writer = PillowWriter(fps=10)
# ani.save("animation.gif", writer=writer)

# from IPython.display import Image
# Image(filename="animation.gif")

### Question 3

3. Replace the semi-implicit scheme for phi with an explicit forward Euler update. What is the largest stable time step you can use before instability sets in? Derive the expected stability condition using dimensional arguments.

---

*Your response below:*
</br>
</br>
$\textbf{Explicit forward Euler update}$
$$\frac{\partial \phi}{\partial t} = -L \left[\frac{\partial f}{\partial \phi} - 2\kappa_{\phi}\nabla^2\phi\right]$$
In fourier space </br>
$$ \hat{\phi}^{n+1} = \hat{\phi}^{n} - L*\Delta t \frac{\partial f}{\partial \hat{\phi}} - 2 * L * \kappa_{\phi} * k^2 * \Delta t * \hat{\phi^{n}} $$
$$ \hat{c}^{n+1} = \hat{c}^n - M \Delta t * k^2 \frac{\partial f}{\partial \hat{c}} - 2.0 * M * \kappa_c * k^4 * \Delta t * \hat{c}^n $$
</br>

- timestep : 0.002 is used. So dt=0.002 can be approximated as the largest time step before instability sets in.
- Ran the simulation for 10000 timestamps but update is slow as compared to the implicit scheme. So will take more timestamps to produce same results as implicit scheme.
- Solution exploding for timestamp > 0.003
- Can't use semi-implicit timestamp=0.1
- Exact stability analysis: Out of scope of this ......

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Define input and model parameters
Nx = 256
Ny = 256
dx = 1.0
dy = 1.0
dt = 0.002
perFrameSteps=100

omega = 1.0
kappa_c = 1.0  # Cahn-Hilliard
kappa_phi = 1.0  # Allen-Cahn 
c0 = 0.1
cAlpha_eq = 0.0
cBeta_eq = 1.0
A = 1.0
B = 1.0
M = 1.0
L = 1.0

# Initialize fields
phi = np.zeros((Nx, Ny))
comp = np.full((Nx, Ny), c0)

# Particle radius and variation
base_radius = 5
radius_variation = 0.01

# Seed the random number generator for reproducibility
np.random.seed(42)
particle_positions = []
radii = []

# Function to calculate the correct distance considering periodic boundaries
def check_distance(x1, y1, r1, x2, y2, r2, Nx, Ny):
    dx = min(abs(x1 - x2), Nx - abs(x1 - x2))
    dy = min(abs(y1 - y2), Ny - abs(y1 - y2))
    distance = np.sqrt(dx**2 + dy**2)
    return distance >= (r1 + r2) * 2  # Safety margin of two times the radius

# Generate 100 unique particles ensuring no overlap within twice the radius
while len(particle_positions) < 100:
    radius = base_radius * (1 + np.random.uniform(-radius_variation, radius_variation))
    x, y = np.random.randint(0, Nx), np.random.randint(0, Ny)

    if all(check_distance(x, y, radius, px, py, r, Nx, Ny) for px, py, r in particle_positions):
        particle_positions.append((x, y, radius))
        radii.append(radius)

# Place particles on the grid
for (x, y, radius) in particle_positions:
    for i in range(-int(radius), int(radius) + 1):
        for j in range(-int(radius), int(radius) + 1):
            if i**2 + j**2 <= radius**2:
                xi = (x + i) % Nx  # Apply periodic boundary conditions
                yj = (y + j) % Ny
                phi[xi, yj] = 1.0
                comp[xi, yj] = cBeta_eq


X,Y = np.meshgrid(range(Nx),range(Ny)) 
fig = plt.figure()
ax = fig.add_subplot()
plt.contourf(X,Y,phi,cmap = 'jet')
ax.set_aspect('equal',adjustable='box')
plt.colorbar()
plt.show()

phi_new = phi
comp_new = comp
#delkx is grid spacing along kx in Fourier space
#delky is grid spacing along ky in Fourier space
delkx = 2*np.pi/(Nx*dx)
delky = 2*np.pi/(Ny*dy)



#Periodic boundar condition in fourier space
kx = np.zeros(Nx)
for i in range(Nx):
    if i < Nx/2:
        kx[i] = i * delkx
    else:
        kx[i] = (i-Nx)*delkx
        
ky = np.zeros(Ny)
for i in range(Ny):
    if i < Ny/2:
        ky[i] = i * delky
    else:
        ky[i] = (i-Ny)*delky
        
kpow2 = np.zeros([Nx,Ny])
kpow4 = np.zeros([Nx,Ny])
for i in range(Nx):
    for j in range(Ny):
        kpow2[i,j] = kx[i]**2 + ky[j]**2
        kpow4[i,j] = kpow2[i,j] * kpow2[i,j]


# Prepare figure for animation
fig, ax = plt.subplots()
c = ax.contourf(comp, levels=50, cmap='jet')
ax.set_aspect('equal')
colorbar = fig.colorbar(c, ax=ax)

# Animation update function
def update(frame):
    global phi, comp, phi_new, comp_new
    for _ in range(perFrameSteps):
        hphi = np.multiply(phi**3,10. - 15. * phi + 6. * phi**2)
        gphi = omega * np.multiply(phi**2,(1-phi)**2)
        hprime = 30.0 * np.multiply(phi**2,(1-phi)**2)
        gprime = 2.0*omega * np.multiply(phi - phi**2,1.0-2.0*phi)
        fAlpha = A * (comp - cAlpha_eq)**2
        fBeta  = B * (cBeta_eq - comp)**2  
    
        dfdc  = 2.0 * A * np.multiply(1.0-hphi,comp-cAlpha_eq)  - 2.0 * B * np.multiply(hphi,cBeta_eq-comp)
        dfdphi = np.multiply(hprime,fBeta-fAlpha) + omega * gprime
    
    
        dfdchat = np.fft.fft2(dfdc)
        dfdphihat = np.fft.fft2(dfdphi)
    
        phi_hat = np.fft.fft2(phi_new)
        # phi_hat = (phi_hat - L*dt*dfdphihat)/(1+2*L*kappa_phi*kpow2*dt)
        phi_hat = phi_hat - L*dt*dfdphihat - 2.0*L*kappa_phi*kpow2*dt*phi_hat
        phi_new = np.fft.ifft2(phi_hat).real
        phi = np.copy(phi_new)
    
        comp_hat = np.fft.fft2(comp_new)
        # comp_hat = (comp_hat - M*dt*kpow2*dfdchat)/(1+2.0*M*kappa_c*kpow4*dt)
        comp_hat = comp_hat - M*dt*kpow2*dfdchat - 2.0*M*kappa_c*kpow4*dt*comp_hat
        comp_new = np.fft.ifft2(comp_hat).real
        comp = np.copy(comp_new)
    
    ax.clear()
    contour = ax.contourf(comp, 50, cmap='jet')
    ax.set_aspect('equal')
    return contour,

# from matplotlib.animation import PillowWriter

# Create and display the animation
ani = FuncAnimation(fig, update, frames=100, repeat=False)
# HTML(ani.to_html5_video())
HTML(ani.to_jshtml())

# Create and display the animation
# ani = FuncAnimation(fig, update, frames=100, repeat=False)
# writer = PillowWriter(fps=10)
# ani.save("animation.gif", writer=writer)

# from IPython.display import Image
# Image(filename="animation.gif")

### Question 4

4. Run a simulation with a biased initial composition, say c0 = 0.6. Does one phase dominate? Why? How does this relate to the shape and depth of the chemical free energy wells?

---

*Your response below:*
- When we change c0 = 0.6 and run, the pahse marked by 'red' dominates. 
- In the final state, there are 3-4 independent particles ('red pahse') as contration is high, so they rate of merging is fast.
- Depth decreases for chemical free energy wells.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Define input and model parameters
Nx = 256
Ny = 256
dx = 1.0
dy = 1.0
dt = 0.1

omega = 1.0
kappa_c = 1.0  # Cahn-Hilliard
kappa_phi = 1.0  # Allen-Cahn 
c0 = 0.6
cAlpha_eq = 0.0
cBeta_eq = 1.0
A = 1.0
B = 1.0
M = 1.0
L = 1.0

# Initialize fields
phi = np.zeros((Nx, Ny))
comp = np.full((Nx, Ny), c0)

# Particle radius and variation
base_radius = 5
radius_variation = 0.01

# Seed the random number generator for reproducibility
np.random.seed(42)
particle_positions = []
radii = []

# Function to calculate the correct distance considering periodic boundaries
def check_distance(x1, y1, r1, x2, y2, r2, Nx, Ny):
    dx = min(abs(x1 - x2), Nx - abs(x1 - x2))
    dy = min(abs(y1 - y2), Ny - abs(y1 - y2))
    distance = np.sqrt(dx**2 + dy**2)
    return distance >= (r1 + r2) * 2  # Safety margin of two times the radius

# Generate 100 unique particles ensuring no overlap within twice the radius
while len(particle_positions) < 100:
    radius = base_radius * (1 + np.random.uniform(-radius_variation, radius_variation))
    x, y = np.random.randint(0, Nx), np.random.randint(0, Ny)

    if all(check_distance(x, y, radius, px, py, r, Nx, Ny) for px, py, r in particle_positions):
        particle_positions.append((x, y, radius))
        radii.append(radius)

# Place particles on the grid
for (x, y, radius) in particle_positions:
    for i in range(-int(radius), int(radius) + 1):
        for j in range(-int(radius), int(radius) + 1):
            if i**2 + j**2 <= radius**2:
                xi = (x + i) % Nx  # Apply periodic boundary conditions
                yj = (y + j) % Ny
                phi[xi, yj] = 1.0
                comp[xi, yj] = cBeta_eq


X,Y = np.meshgrid(range(Nx),range(Ny)) 
fig = plt.figure()
ax = fig.add_subplot()
plt.contourf(X,Y,phi,cmap = 'jet')
ax.set_aspect('equal',adjustable='box')
plt.colorbar()
plt.show()

phi_new = phi
comp_new = comp
#delkx is grid spacing along kx in Fourier space
#delky is grid spacing along ky in Fourier space
delkx = 2*np.pi/(Nx*dx)
delky = 2*np.pi/(Ny*dy)



#Periodic boundar condition in fourier space
kx = np.zeros(Nx)
for i in range(Nx):
    if i < Nx/2:
        kx[i] = i * delkx
    else:
        kx[i] = (i-Nx)*delkx
        
ky = np.zeros(Ny)
for i in range(Ny):
    if i < Ny/2:
        ky[i] = i * delky
    else:
        ky[i] = (i-Ny)*delky
        
kpow2 = np.zeros([Nx,Ny])
kpow4 = np.zeros([Nx,Ny])
for i in range(Nx):
    for j in range(Ny):
        kpow2[i,j] = kx[i]**2 + ky[j]**2
        kpow4[i,j] = kpow2[i,j] * kpow2[i,j]


# Prepare figure for animation
fig, ax = plt.subplots()
c = ax.contourf(comp, levels=50, cmap='jet')
ax.set_aspect('equal')
colorbar = fig.colorbar(c, ax=ax)

# Animation update function
def update(frame):
    global phi, comp, phi_new, comp_new
    for _ in range(100):
        hphi = np.multiply(phi**3,10. - 15. * phi + 6. * phi**2)
        gphi = omega * np.multiply(phi**2,(1-phi)**2)
        hprime = 30.0 * np.multiply(phi**2,(1-phi)**2)
        gprime = 2.0*omega * np.multiply(phi - phi**2,1.0-2.0*phi)
        fAlpha = A * (comp - cAlpha_eq)**2
        fBeta  = B * (cBeta_eq - comp)**2  
    
        dfdc  = 2.0 * A * np.multiply(1.0-hphi,comp-cAlpha_eq)  - 2.0 * B * np.multiply(hphi,cBeta_eq-comp)
        dfdphi = np.multiply(hprime,fBeta-fAlpha) + omega * gprime
    
    
        dfdchat = np.fft.fft2(dfdc)
        dfdphihat = np.fft.fft2(dfdphi)
    
        phi_hat = np.fft.fft2(phi_new)
        phi_hat = (phi_hat - L*dt*dfdphihat)/(1+2*L*kappa_phi*kpow2*dt)
        phi_new = np.fft.ifft2(phi_hat).real
        phi = np.copy(phi_new)
    
        comp_hat = np.fft.fft2(comp_new)
        comp_hat = (comp_hat - M*dt*kpow2*dfdchat)/(1+2.0*M*kappa_c*kpow4*dt)
        comp_new = np.fft.ifft2(comp_hat).real
        comp = np.copy(comp_new)
    
    ax.clear()
    contour = ax.contourf(comp, 50, cmap='jet')
    ax.set_aspect('equal')
    return contour,

# from matplotlib.animation import PillowWriter

# Create and display the animation
ani = FuncAnimation(fig, update, frames=100, repeat=False)
# HTML(ani.to_html5_video())
HTML(ani.to_jshtml())

# Create and display the animation
# ani = FuncAnimation(fig, update, frames=100, repeat=False)
# writer = PillowWriter(fps=10)
# ani.save("animation.gif", writer=writer)

# from IPython.display import Image
# Image(filename="animation.gif")